Data Preprocessing

In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import os

# Load your dataset
file_path = r'D:/Guvi/Own_Projects/Sentiment_Analysis_on_Social_Media/twitter_training.csv'
df = pd.read_csv(file_path)

# Rename columns for clarity
df.columns = ['ID', 'Category', 'Sentiment', 'Text']

# Drop any rows where 'Text' is null
df_cleaned = df.dropna(subset=['Text'])

# Drop duplicates
df_cleaned = df_cleaned.drop_duplicates(subset=['Text'])

# Define manual stopwords
manual_stopwords = {
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
    'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it',
    "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',
    'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before',
    'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
    'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
    'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
    'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren',
    "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
    "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
    'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
}

# Function to clean text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters
    text = ' '.join(word for word in text.split() if word not in manual_stopwords)  # Remove stopwords
    return text

# Apply text cleaning
df_cleaned['Cleaned_Text'] = df_cleaned['Text'].apply(clean_text)

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Vectorize the cleaned text
X = vectorizer.fit_transform(df_cleaned['Cleaned_Text']).toarray()

# Encode the sentiment labels (assuming 'Positive' is 1 and others are 0)
y = df_cleaned['Sentiment'].apply(lambda x: 1 if x == 'Positive' else 0)

# Save the vectorizer for future use in deployment
joblib.dump(vectorizer, os.path.join(os.path.dirname(file_path), 'tfidf_vectorizer.pkl'))

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save the preprocessed data with compression to save space
joblib.dump((X_train, X_test, y_train, y_test), os.path.join(os.path.dirname(file_path), 'preprocessed_data.pkl'), compress=3)


['D:/Guvi/Own_Projects/Sentiment_Analysis_on_Social_Media\\preprocessed_data.pkl']

 Model Training

In [6]:


import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split

# Load preprocessed data
file_path = r'D:/Guvi/Own_Projects/Sentiment_Analysis_on_Social_Media/preprocessed_data.pkl'
X_train, X_test, y_train, y_test = joblib.load(file_path)

# If memory issues persist, you can use a smaller subset of the training data
# X_train, _, y_train, _ = train_test_split(X_train, y_train, train_size=0.5, random_state=42)

# Initialize the Random Forest model with optimized parameters
model = RandomForestClassifier(
    n_estimators=50,            # Reduce number of trees
    max_depth=20,               # Limit the depth of trees
    min_samples_split=10,       # Minimum samples required to split an internal node
    n_jobs=-1,                  # Utilize all available cores
    random_state=42
)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

# Save the trained model
model_save_path = os.path.join(os.path.dirname(file_path), 'sentiment_rf_model_optimized.pkl')
joblib.dump(model, model_save_path)


Accuracy: 0.7337026910346812
Classification Report:
              precision    recall  f1-score   support

           0       0.73      1.00      0.84     10076
           1       0.98      0.03      0.06      3822

    accuracy                           0.73     13898
   macro avg       0.85      0.52      0.45     13898
weighted avg       0.80      0.73      0.63     13898



['D:/Guvi/Own_Projects/Sentiment_Analysis_on_Social_Media\\sentiment_rf_model_optimized.pkl']

In [ ]:
1) Data Preprocessing :                                                                                                                    import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import os

# Load your dataset
file_path = r'D:/Guvi/Own_Projects/Sentiment_Analysis_on_Social_Media/twitter_training.csv'
df = pd.read_csv(file_path)

# Rename columns for clarity
df.columns = ['ID', 'Category', 'Sentiment', 'Text']

# Drop any rows where 'Text' is null
df_cleaned = df.dropna(subset=['Text'])

# Drop duplicates
df_cleaned = df_cleaned.drop_duplicates(subset=['Text'])

# Define manual stopwords
manual_stopwords = {
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
    'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it',
    "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',
    'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before',
    'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
    'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
    'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
    'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren',
    "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
    "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
    'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
}
# Function to clean text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters
    text = ' '.join(word for word in text.split() if word not in manual_stopwords)  # Remove stopwords
    return text

# Apply text cleaning
df_cleaned['Cleaned_Text'] = df_cleaned['Text'].apply(clean_text)

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)

# Vectorize the cleaned text
X = vectorizer.fit_transform(df_cleaned['Cleaned_Text']).toarray()

# Encode the sentiment labels (assuming 'Positive' is 1 and others are 0)
y = df_cleaned['Sentiment'].apply(lambda x: 1 if x == 'Positive' else 0)

# Save the vectorizer for future use in deployment
joblib.dump(vectorizer, os.path.join(os.path.dirname(file_path), 'tfidf_vectorizer.pkl'))

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Save the preprocessed data with compression to save space
joblib.dump((X_train, X_test, y_train, y_test), os.path.join(os.path.dirname(file_path), 'preprocessed_data.pkl'), compress=3)                                                                                           2) model training :                                                                                                                  import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_selection import SelectFromModel
import os

# Load preprocessed data
file_path = r'D:/Guvi/Own_Projects/Sentiment_Analysis_on_Social_Media/preprocessed_data.pkl'
X_train, X_test, y_train, y_test = joblib.load(file_path)

# Feature selection using Random Forest's feature importance
feature_selector = SelectFromModel(RandomForestClassifier(n_estimators=50, random_state=42))
X_train_selected = feature_selector.fit_transform(X_train, y_train)
X_test_selected = feature_selector.transform(X_test)

# Initialize the Random Forest model with optimized parameters
model = RandomForestClassifier(
    n_estimators=40,            # Reduce number of trees slightly
    max_depth=25,               # Slightly increase the depth
    min_samples_split=10,       # Minimum samples required to split an internal node
    min_samples_leaf=5,         # Increase min samples per leaf to prevent overfitting
    max_features='sqrt',        # Use the square root of the number of features
    n_jobs=-1,                  # Utilize all available cores
    random_state=42
)

# Stratified K-Fold Cross-Validation for better generalization (optional)
 skf = StratifiedKFold(n_splits=3)
 for train_index, val_index in skf.split(X_train_selected, y_train):
     X_train_fold, X_val_fold = X_train_selected[train_index], X_train_selected[val_index]
     y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
     model.fit(X_train_fold, y_train_fold)
     val_pred = model.predict(X_val_fold)
     print(f"Validation Accuracy: {accuracy_score(y_val_fold, val_pred)}")

# Train the model on the full training data
model.fit(X_train_selected, y_train)

# Make predictions
y_pred = model.predict(X_test_selected)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n{report}")

# Save the trained model
model_save_path = os.path.join(os.path.dirname(file_path), 'sentiment_rf_model_optimized.pkl')
joblib.dump(model, model_save_path)
 3) deployment Stream lit app :                                                                                                     import streamlit as st
import joblib
import pandas as pd
import os
import re

# Function to clean text
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters
    text = ' '.join(word for word in text.split() if word not in manual_stopwords)  # Remove stopwords
    return text

# Load the TF-IDF vectorizer and the trained model
dataset_path = r'D:/Guvi/Own_Projects/Sentiment_Analysis_on_Social_Media/twitter_training.csv'
data_directory = os.path.dirname(dataset_path)

vectorizer_path = os.path.join(data_directory, 'tfidf_vectorizer.pkl')
model_path = os.path.join(data_directory, 'sentiment_model.pkl')

vectorizer = joblib.load(vectorizer_path)
model = joblib.load(model_path)

# Define manual stopwords
manual_stopwords = {
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your',
    'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it',
    "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this',
    'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until',
    'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before',
    'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',
    'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most',
    'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can',
    'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren',
    "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven',
    "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't",
    'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"
}

# Streamlit app layout
st.set_page_config(page_title="Sentiment Analysis", page_icon=":sparkles:", layout="wide")

# CSS for styling
st.markdown("""
    <style>
        .main {
            color: #333;
            background-color: #f5f5f5;
            padding: 20px;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }
        .input-box {
            background-color: #ffffff;
            border: 1px solid #ddd;
            border-radius: 8px;
            padding: 10px;
            margin-bottom: 20px;
        }
        .text-output {
            padding: 15px;
            border-radius: 8px;
            color: #ffffff;
        }
        .positive {
            background-color: #4CAF50;
        }
        .negative {
            background-color: #F44336;
        }
        .stButton > button {
            background-color: #007BFF;
            color: white;
            border-radius: 8px;
            padding: 10px;
            border: none;
        }
        .stButton > button:hover {
            background-color: #0056b3;
        }
    </style>
""", unsafe_allow_html=True)

# Title and description
st.markdown("<div class='main'><h1 style='text-align:center;'>🗣️ Sentiment Analysis Tool</h1></div>", unsafe_allow_html=True)
st.markdown("""
    <div class='main'>
    <p style='text-align:center;'>Welcome to the **Sentiment Analysis Tool**! 🎉</p>
    <p style='text-align:center;'>Enter your text below to get sentiment analysis results.</p>
    </div>
""", unsafe_allow_html=True)

# Text input
st.markdown("<div class='input-box'><h3>Input Text:</h3></div>", unsafe_allow_html=True)
text_input = st.text_area("", "Type something...", height=150)

# Button to analyze sentiment
if st.button("Analyze Sentiment"):
    if text_input.strip() == "":
        st.error("Please enter some text before analyzing.")
    else:
        # Clean and vectorize the input text
        cleaned_text = clean_text(text_input)
        vectorized_text = vectorizer.transform([cleaned_text]).toarray()

        # Make a prediction
        prediction = model.predict(vectorized_text)[0]
        sentiment = "Positive 😊" if prediction == 1 else "Negative 😔"
        sentiment_class = "positive" if prediction == 1 else "negative"

        # Display the result
        st.markdown(f"""
            <div class='text-output {sentiment_class}'>
                <h2 style='text-align:center;'>{sentiment}</h2>
            </div>
            <div class='main'>
                <h3>Your Text:</h3>
                <p>{text_input}</p>
            </div>
        """, unsafe_allow_html=True)

use all this codes and 1) create a detailed report on 'Sentiment Analysis on Social media using Machine learning' add user guide for end users. 2) create a readme file in format and short description for 3 lines to for my git hub repository . 3) write a post in linked in on 'Sentiment Analysis on Social media using Machine learning' on which im going to add my github repository  